# ACGS-2 Policy Experimentation Notebook

Welcome to the ACGS-2 Policy Experimentation Notebook! This interactive notebook allows you to:

- Write and test Rego policies in real-time
- Experiment with different governance scenarios
- Visualize policy evaluation results
- Learn Rego syntax through hands-on examples

## Getting Started

This notebook connects to your local ACGS-2 instance. Make sure you have ACGS-2 running:

```bash
cd acgs2-core
docker-compose up -d
```

In [ ]:
# Setup and imports
import requests
import json
from IPython.display import JSON, HTML, Markdown
import ipywidgets as widgets
from IPython.display import display, clear_output

# ACGS-2 service URLs
API_GATEWAY_URL = "http://localhost:8080"
AGENT_BUS_URL = "http://localhost:8000"
OPA_URL = "http://localhost:8181"

print("ACGS-2 Policy Experimentation Notebook")
print("======================================")
print(f"API Gateway: {API_GATEWAY_URL}")
print(f"Agent Bus: {AGENT_BUS_URL}")
print(f"OPA: {OPA_URL}")

In [ ]:
# Health check function
def check_services():
    """Check if ACGS-2 services are running"""
    services = {
        "API Gateway": f"{API_GATEWAY_URL}/health",
        "Agent Bus": f"{AGENT_BUS_URL}/health",
        "OPA": f"{OPA_URL}/health"
    }

    results = {}
    for name, url in services.items():
        try:
            response = requests.get(url, timeout=5)
            if response.status_code == 200:
                results[name] = "✅ Healthy"
            else:
                results[name] = f"⚠️ Status {response.status_code}"
        except Exception as e:
            results[name] = f"❌ Error: {str(e)}"

    return results

# Check services
service_status = check_services()
for service, status in service_status.items():
    print(f"{service}: {status}")

all_healthy = all("✅" in status for status in service_status.values())
if all_healthy:
    print("\n🎉 All services are running! Ready to experiment.")
else:
    print("\n⚠️ Some services are not running. Please start ACGS-2 first.")

## Experiment 1: Basic Policy Evaluation

Let's start with a simple policy that checks if content is appropriate for a general audience.

In [ ]:
# Sample policy
sample_policy = '''
package acgs2.experiment.basic

# Basic content appropriateness policy
default allow = false

allow {
    input.content_type == "educational"
    not contains_inappropriate(input.content)
}

contains_inappropriate(content) {
    bad_words := ["inappropriate", "offensive", "spam"]
    some word in bad_words
    contains(lower(content), lower(word))
}
'''

print("Sample Policy:")
print(sample_policy)

In [ ]:
# Test data
test_cases = [
    {
        "description": "Educational content - should be allowed",
        "input": {
            "content_type": "educational",
            "content": "Python is a programming language used for data science."
        }
    },
    {
        "description": "Inappropriate content - should be blocked",
        "input": {
            "content_type": "educational",
            "content": "This contains inappropriate content."
        }
    },
    {
        "description": "Non-educational content - should be blocked",
        "input": {
            "content_type": "entertainment",
            "content": "This is just entertainment content."
        }
    }
]

# Display test cases
for i, test in enumerate(test_cases, 1):
    print(f"Test {i}: {test['description']}")
    print(f"Input: {json.dumps(test['input'], indent=2)}")
    print()

In [ ]:
# Function to evaluate policy against test data
def evaluate_policy_locally(policy_code, input_data):
    """Simple local policy evaluation (mock implementation)"""
    # This is a simplified evaluation - in real ACGS-2, this would go through OPA
    content_type = input_data.get("content_type", "")
    content = input_data.get("content", "").lower()

    # Check content type
    if content_type != "educational":
        return {"allow": False, "reason": "Content type must be educational"}

    # Check for inappropriate content
    bad_words = ["inappropriate", "offensive", "spam"]
    for word in bad_words:
        if word in content:
            return {"allow": False, "reason": f"Content contains '{word}'"}

    return {"allow": True, "reason": "Content approved"}

# Run tests
print("Running Policy Tests:")
print("=" * 50)

for i, test in enumerate(test_cases, 1):
    result = evaluate_policy_locally(sample_policy, test["input"])
    status = "✅ ALLOWED" if result["allow"] else "❌ BLOCKED"
    print(f"Test {i}: {status}")
    print(f"  Reason: {result['reason']}")
    print(f"  Input: {test['input']}")
    print()

## Experiment 2: Intent Classification Integration

Now let's experiment with ACGS-2's intent classification feature.

In [ ]:
# Test intent classification
def classify_intent(text):
    """Classify intent using ACGS-2 Agent Bus"""
    try:
        response = requests.post(
            f"{AGENT_BUS_URL}/api/v1/intent/classify",
            json={"text": text, "context": []},
            timeout=10
        )
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"HTTP {response.status_code}", "fallback": "neutral"}
    except Exception as e:
        return {"error": str(e), "fallback": "neutral"}

# Test various inputs
test_texts = [
    "How do I write a Python function?",
    "This product is terrible and doesn't work",
    "Tell me a joke",
    "I need help with my computer that's broken",
    "Buy this amazing product right now!"
]

print("Intent Classification Results:")
print("=" * 40)

for text in test_texts:
    result = classify_intent(text)
    if "error" in result:
        print(f"❌ Error classifying: {text}")
        print(f"   {result['error']}")
    else:
        classification = result.get("classification", "unknown")
        confidence = result.get("confidence", 0)
        print(f"📝 '{text[:50]}...'")
        print(f"   Classification: {classification} (confidence: {confidence:.2f})")
    print()

## Experiment 3: Advanced Policy with Intent Integration

Let's create a policy that combines content checking with intent classification.

In [ ]:
# Advanced policy combining intent and content
advanced_policy_tests = [
    {
        "description": "Helpful educational content",
        "text": "Can you explain how neural networks work?",
        "expected": "Should be allowed"
    },
    {
        "description": "Harmful intent",
        "text": "How can I hack into someone's computer?",
        "expected": "Should be blocked"
    },
    {
        "description": "Spam content",
        "text": "BUY NOW!!! Amazing product that will change your life!!!",
        "expected": "Should be blocked"
    }
]

def evaluate_advanced_policy(text):
    """Evaluate content using both intent classification and content analysis"""
    # Get intent classification
    intent_result = classify_intent(text)

    if "error" in intent_result:
        return {"allow": False, "reason": "Could not classify intent", "details": intent_result}

    classification = intent_result.get("classification", "unknown")
    confidence = intent_result.get("confidence", 0.0)

    # Policy logic
    if classification == "helpful" and confidence > 0.8:
        # Check for spam-like content even in helpful messages
        spam_indicators = ["BUY NOW", "!!!", "AMAZING", "CHANGE YOUR LIFE"]
        if any(indicator.upper() in text.upper() for indicator in spam_indicators):
            return {
                "allow": False,
                "reason": "Helpful but contains spam indicators",
                "intent": intent_result
            }
        return {
            "allow": True,
            "reason": "Helpful content approved",
            "intent": intent_result
        }

    if classification == "harmful":
        return {
            "allow": False,
            "reason": "Harmful intent detected",
            "intent": intent_result
        }

    return {
        "allow": False,
        "reason": f"Policy violation: {classification} (confidence: {confidence:.2f})",
        "intent": intent_result
    }

# Run advanced policy tests
print("Advanced Policy Evaluation:")
print("=" * 50)

for i, test in enumerate(advanced_policy_tests, 1):
    print(f"Test {i}: {test['description']}")
    print(f"Text: {test['text']}")
    print(f"Expected: {test['expected']}")

    result = evaluate_advanced_policy(test["text"])
    status = "✅ ALLOWED" if result["allow"] else "❌ BLOCKED"
    print(f"Result: {status}")
    print(f"Reason: {result['reason']}")

    if "intent" in result:
        intent = result["intent"]
        if "classification" in intent:
            print(f"Intent: {intent['classification']} ({intent.get('confidence', 0):.2f})")

    print("-" * 50)
    print()

## Experiment 4: Build Your Own Policy

Now it's your turn! Use the cells below to create and test your own governance policy.

In [ ]:
# Create your own policy here
my_policy_name = "My Custom Policy"
my_policy_description = "A policy I created for learning ACGS-2"

# Define your policy logic
def my_custom_policy(input_data):
    """
    Your custom policy logic here.

    Input data will contain:
    - text: the content to evaluate
    - intent: classification results (if available)
    - metadata: additional context

    Return a dict with:
    - allow: boolean
    - reason: string explanation
    """

    text = input_data.get("text", "").lower()
    intent = input_data.get("intent", {})

    # Example: Block all messages containing certain words
    blocked_words = ["blocked", "forbidden", "not allowed"]

    for word in blocked_words:
        if word in text:
            return {
                "allow": False,
                "reason": f"Content contains blocked word: '{word}'"
            }

    # Allow everything else
    return {
        "allow": True,
        "reason": "Content approved by custom policy"
    }

print(f"Policy: {my_policy_name}")
print(f"Description: {my_policy_description}")
print("\nReady to test your policy!")
print("Edit the my_custom_policy function above, then run the next cell.")

In [ ]:
# Test your custom policy
my_test_cases = [
    {
        "text": "This is a normal message",
        "intent": {"classification": "neutral", "confidence": 0.8}
    },
    {
        "text": "This message contains blocked content",
        "intent": {"classification": "neutral", "confidence": 0.8}
    },
    {
        "text": "This is forbidden",
        "intent": {"classification": "harmful", "confidence": 0.9}
    }
]

print("Testing Your Custom Policy:")
print("=" * 50)

for i, test in enumerate(my_test_cases, 1):
    result = my_custom_policy(test)
    status = "✅ ALLOWED" if result["allow"] else "❌ BLOCKED"
    print(f"Test {i}: {status}")
    print(f"  Text: {test['text']}")
    print(f"  Reason: {result['reason']}")
    print()

## Next Steps

Congratulations! You've completed the ACGS-2 Policy Experimentation Notebook. Here's what you learned:

1. ✅ **Basic Policy Evaluation** - How to write and test simple Rego policies
2. ✅ **Intent Classification** - How ACGS-2 classifies content intent
3. ✅ **Advanced Policies** - Combining intent with content analysis
4. ✅ **Custom Policy Creation** - Building your own governance logic

## What's Next?

- **Deploy Real Policies**: Copy your policies to `acgs2-core/policies/rego/` and deploy them
- **Explore Examples**: Check out the example projects in `acgs2-core/examples/`
- **Read the Docs**: Visit https://docs.acgs2.dev for detailed documentation
- **Join the Community**: Connect with other ACGS-2 users at https://community.acgs2.dev

## Need Help?

- 📚 **Documentation**: https://docs.acgs2.dev
- 💬 **Community**: https://community.acgs2.dev
- 🐛 **Issues**: https://github.com/acgs2/acgs2-core/issues
- 📧 **Support**: support@acgs2.dev

Happy governing! 🎉